In [ ]:
from sympy.physics.units import *
from sympy import *

# Rounding:
import decimal
from decimal import Decimal as DX
def iso_round(obj, pv, rounding=decimal.ROUND_HALF_EVEN):
    import sympy
    """
    Rounding acc. to DIN EN ISO 80000-1:2013-08
    place value = Rundestellenwert
    """
    assert pv in set([
        # place value   #  round to:
        100,            #  3rd last digit before decimal
        10,             #  2nd last
        1,              #  last
        0.1,            #  1st digit after decimal
        0.01,           #  2nd
        0.001,          #  3rd
        0.0001,         #  4th
        0.00001,        #  5th
        0.000001,       #  6th
        0.0000001,      #  7th
        0.00000001,     #  8th
        0.000000001,    #  9th
        0.0000000001,   # 10th
        ])
    try:
        tmp = DX(str(float(obj)))
        obj = tmp.quantize(DX(str(pv)), rounding=rounding)
    except:
        for i in range(len(obj)):
            tmp = DX(str(float(obj[i])))
            obj[i] = tmp.quantize(DX(str(pv)), rounding=rounding)
    return obj

# LateX:
kwargs = {}
kwargs["mat_str"] = "bmatrix"
kwargs["mat_delim"] = ""
# kwargs["symbol_names"] = {FB: "F^{\mathsf B}", }

# Units:
(k, M, G ) = ( 10**3, 10**6, 10**9 )
(mm, cm, deg) = ( m/1000, m/100, pi/180)
Newton = kg*m/s**2
Pa     = Newton/m**2
MPa    = M*Pa
GPa    = G*Pa
kN     = k*Newton

# ---

F1, F2, l, EI = var("F1, F2, l, EI")

sub_list = [
    (F1,   10 *kN             ),
    (F2,   10 *kN             ),
    (l,     1 *m              ),
    (EI,  210 *GPa * 318*cm**4), # IPE 120
    ]

l2 = l*l
l3 = l*l*l

K = EI/l3
K *= Matrix(
[
[  4*l2 ,  -6*l ,  2*l2 ,   6*l ,  0    ,   0    ],
[ -6*l  ,  12   , -6*l  , -12   ,  0    ,   0    ],
[  2*l2 ,  -6*l ,  8*l2 ,   0   ,  2*l2 ,   6*l  ],
[  6*l  , -12   ,  0    ,  24   , -6*l  , -12    ],
[  0    ,   0   ,  2*l2 ,  -6*l ,  4*l2 ,   6*l  ],
[  0    ,   0   ,  6*l  , -12   ,  6*l  ,  12    ],
]
)
# pprint(K)

p0,w0,p1,w1,p2,w2 = var("p0,w0,p1,w1,p2,w2")
M0,Q0,M1,Q1,M2,Q2 = var("M0,Q0,M1,Q1,M2,Q2")
u = Matrix([p0,w0,p1,w1,p2,w2])
f = Matrix([M0,Q0,M1,Q1,M2,Q2])

unknowns = [p0,w0,p1,w1,p2,w2, M0,Q0,M1,Q1,M2,Q2]

# boundary conditions:
# --- a ---
sub_list_u=[
(p0, 0),
(w0, 0),
]
sub_list_f=[
(M1, 0),
(Q1, F1),
(M2, 0),
(Q2, F2),
]


u = u.subs(sub_list_u)
f = f.subs(sub_list_f)
eq = Eq(K*u , f)
sol = solve(eq, unknowns)
for s in sol:
    pprint("\n")
    pprint(s)
    pprint(sol[s])

w2 = sol[w2]
pprint("\nw2 / mm:")
tmp = w2.subs(sub_list)
tmp /= mm
tmp = tmp.simplify()
tmp = iso_round(tmp,0.01)
pprint(tmp)

# M₁
# l⋅(F₁ + 2⋅F₂)
#
# u₂
#  3
# l ⋅(2⋅F₁ + 5⋅F₂)
# ────────────────
#       6⋅EI
#
# p₂
#   2
# -l ⋅(F₁ + 3⋅F₂)
# ────────────────
#       2⋅EI
#
# u₃
#  3
# l ⋅(5⋅F₁ + 16⋅F₂)
# ─────────────────
#        6⋅EI
#
# Q₁
# -F₁ - F₂
#
# p₃
#   2
# -l ⋅(F₁ + 4⋅F₂)
# ────────────────
#       2⋅EI
